# Trainerを使ってみよう

`Trainer`を使うと学習ループを明示的に書く必要がなくなり、またいろいろな便利なExtentionを使うことで可視化や`Logging`などが楽になります。

まずは必要なものを`import`しておきます。

In [1]:
import chainer
from chainer import training
from chainer.training import extensions
from chainer.datasets import mnist

## データセットの準備

In [2]:
train, test = mnist.get_mnist()

## Iteratorの準備

In [3]:
batchsize = 128

train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter = chainer.iterators.SerialIterator(
    test, batchsize, repeat=False, shuffle=False)

## Modelの準備

In [4]:
# ここは不要（赤いWarningが出て見にくいので出ないようにしている）
import warnings
warnings.filterwarnings('ignore')

# ここから必要
import sys
sys.path.insert(0, 'chainer/examples/mnist')
from train_mnist import MLP

model = MLP(n_units=100, n_out=10)

# Updaterの準備

Trainerは学習に必要な全てのものをひとまとめにするクラスです。それは主に以下のようなものを保持します。

- Updater
    - Iterator
    - Optimizer
        - Model

`Trainer`オブジェクトを作成するときに渡すのは基本的に`Updater`だけですが、`Updater`は中に`Iterator`と`Optimizer`を持っています。`Iterator`からはデータセットにアクセスすることができ、`Optimizer`は中でモデルへの参照を保持しているので、モデルのパラメータを更新することができます。つまり、`Updater`が内部で

1. データセットからデータを取り出し
2. モデルに渡してロスを計算し
3. Optimizerを使ってモデルのパラメータを更新する

という一連の学習の主要部分を行うことができるということです。では、`Updater`オブジェクトを作成してみます。

In [5]:
import numpy as np
import chainer.functions as F
import chainer.links as L

max_epoch = 10
gpu_id = 0

# モデルをClassifierで包んで、ロスの計算などをモデルに含める
model = L.Classifier(model)
model.to_gpu(gpu_id)

# 最適化手法の選択
optimizer = chainer.optimizers.SGD()
optimizer.setup(model)

# UpdaterにIteratorとOptimizerを渡す
updater = training.StandardUpdater(train_iter, optimizer, device=gpu_id)

# Trainerの設定

最後に、`Trainer`の設定を行います。`Trainer`のオブジェクトを作成する際に必須となるのは、先程作成した`Updater`オブジェクトだけですが、二番目の引数に学習をどのタイミングで終了するかを指定する`stop_trigger`を与えると、最大エポック数または最大イテレーション数を指定することができます。

In [6]:
# TrainerにUpdaterを渡す
trainer = training.Trainer(updater, (max_epoch, 'epoch'))

## TrainerにExtensionを追加する


`Trainer`を使う利点として、

- ログを自動的にファイルに保存（`LogReport`)
- ターミナルに定期的にロスなどの情報を表示（`PrintReport`）
- ロスを定期的にグラフで可視化して画像として保存（`PlotReport`)
- 定期的にモデルやOptimizerの状態を自動シリアライズ（`snapshot`/`snapshot_object`）
- 学習の進捗を示すプログレスバーを表示（`ProgressBar`）
- モデルの構造をGraphvizのdot形式で保存（`dump_graph`）

などなどの様々な便利な機能を利用することができる点があります。これらの機能を利用するのは簡単で、`Trainer`オブジェクトに対して`extend`メソッドを使って追加したい`Extension`のオブジェクトを渡してやるだけです。では実際に幾つかの`Extension`を追加してみましょう。

In [7]:
from chainer.training import extensions

trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'main/accuracy', 'elapsed_time']))

trainer.run()

epoch       main/loss   main/accuracy  elapsed_time
1           1.44393     0.644856       2.63335       
2           0.570543    0.857659       4.51591       
3           0.414136    0.888409       6.42211       
4           0.358988    0.900407       8.29233       
5           0.328933    0.907116       10.157        
6           0.308216    0.912197       12.1324       
7           0.291617    0.917627       14.1007       
8           0.278497    0.920807       15.902        
9           0.266466    0.924091       17.8218       
10          0.255959    0.927322       19.7502       
